In [1]:
#importa a biblioteca do Gurobi
import gurobipy as gp

In [29]:
#parâmetros do problema
qntd_caixas = 4
capacidade_helicoptero = 7

#vetor com as caixas
vet_caixas = ['alimento', 'água', 'munição', 'remédio/soro']

#quantidade de caixas para a sobrevivência dos tripulantes
caixa_sobrevivencia = [6, 4, 2, 2]

#vetor com o valor de importância das caixas de emergência
#assumindo que as caixas de munição e remedio são 2 vezes mais importantes do que as de água 
#e 4 vezes mais importantes do que as de alimento
vet_pontos = [0.5, 1, 2, 2]

#rótulo das caixas de emergência
caixa = list()
for i in range(qntd_caixas):
    caixa.append("Caixa {}".format(i+1))

#dicionário das caixas de emergência
dic_caixas = dict()
for i in range(qntd_caixas):
    rot_caixa = caixa[i]
    dic_caixas[rot_caixa] = vet_caixas[i]

#dicionário com a quantidade de caixas para a sobrevivência dos tripulantes
dic_caixa_sobrev = dict()
for i in range(qntd_caixas):
    rot_caixa = caixa[i]
    dic_caixa_sobrev[rot_caixa] = caixa_sobrevivencia[i]

#dicionário com o valor de importância das caixas de emergência
dic_caixas_pontos = dict()
for i in range(qntd_caixas):
    rot_caixa = caixa[i]
    dic_caixas_pontos[rot_caixa] = vet_pontos[i]
    

In [32]:
#criando o modelo
m = gp.Model("Problema do Pedido de Socorro")
m.setParam(gp.GRB.Param.OutputFlag, 0)

#variáveis de decisão
x = m.addVars(caixa, vtype=gp.GRB.INTEGER)

#restrições:
#1)quanto a capacidade do helicoptéro 
r1 = m.addConstrs(gp.quicksum(x[i] for i in caixa) <= capacidade_helicoptero for i in caixa)

#2)quanto a quantidade de caixas para a sobrevivência dos tripulantes
r2 = m.addConstrs(x[i] <= dic_caixa_sobrev[i] for i in caixa )

#função objetivo
m.setObjective(gp.quicksum(x[i] * dic_caixas_pontos[i] for i in caixa),
                sense=gp.GRB.MAXIMIZE
)


In [33]:
#executa o modelo
m.optimize()

# valor da função objetivo
print("Função objetivo: ", m.objVal, "\n")

#total de caixas a serem transportadas
for i in caixa:
    print("Será tranportada {} de {}".format(round(x[i].X), dic_caixas[i]))

Função objetivo:  11.0 

Será tranportada 0 de alimento
Será tranportada 3 de água
Será tranportada 2 de munição
Será tranportada 2 de remédio/soro
